In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df_original = pd.read_csv('/content/drive/MyDrive/CO544/transformed_rainfall_data.csv')

Mounted at /content/drive


In [2]:
# Assuming your dataframe is named 'df'
df = df_original[~df_original.index.isin([0, 1])]

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming your data is loaded into a DataFrame named 'df'

# Convert the date column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Extracting day of the year as a feature
df['DayOfYear'] = df['Date'].dt.dayofyear

# Select features and target
features = ['DayOfYear']
targets = ['Vavuniya', 'Anuradhapura', 'Maha Illuppallama']

# Feature engineering

# Lag features (previous day's value)
for target in targets:
  df[f'{target}_lag1'] = df[target].shift(1)

# Rolling mean (weekly window)
window_size = 7  # You can experiment with different window sizes
for target in targets:
  df[f'{target}_rolling_mean_{window_size}'] = df[target].rolling(window=window_size).mean()

# Month and Weekday features
# df['Month'] = df['Date'].dt.month
# df['Is_Weekend'] = np.where(df['Date'].dt.weekday_name.isin(['Saturday', 'Sunday']), 1, 0)

# After feature engineering
df.dropna(inplace=True)  # Drop rows with any NaN values

# Splitting the data into training and testing sets
X = df[features + [col for col in df.columns if col.endswith('_lag1') or col.endswith(f'_rolling_mean_{window_size}')]]  # Include engineered features
y_vavuniya = df['Vavuniya']
y_anuradhapura = df['Anuradhapura']
y_maha = df['Maha Illuppallama']



X_train, X_test, y_train_v, y_test_v = train_test_split(X, y_vavuniya, test_size=0.2, random_state=42)
X_train, X_test, y_train_a, y_test_a = train_test_split(X, y_anuradhapura, test_size=0.2, random_state=42)
X_train, X_test, y_train_m, y_test_m = train_test_split(X, y_maha, test_size=0.2, random_state=42)

# ... (rest of your code for training and prediction)


<ipython-input-3-ab9b7845a373>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
<ipython-input-3-ab9b7845a373>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DayOfYear'] = df['Date'].dt.dayofyear
<ipython-input-3-ab9b7845a373>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [5]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

xgb_v = XGBRegressor()
xgb_a = XGBRegressor()
xgb_m = XGBRegressor()

# Fit the models
xgb_v.fit(X_train, y_train_v)
xgb_a.fit(X_train, y_train_a)
xgb_m.fit(X_train, y_train_m)

# Predict and evaluate
y_pred_v_xgb = xgb_v.predict(X_test)
y_pred_a_xgb = xgb_a.predict(X_test)
y_pred_m_xgb = xgb_m.predict(X_test)

mse_v_xgb = mean_squared_error(y_test_v, y_pred_v_xgb)
mse_a_xgb = mean_squared_error(y_test_a, y_pred_a_xgb)
mse_m_xgb = mean_squared_error(y_test_m, y_pred_m_xgb)

print(f'XGBoost Regressor MSE for Vavuniya: {mse_v_xgb}')
print(f'XGBoost Regressor MSE for Anuradhapura: {mse_a_xgb}')
print(f'XGBoost Regressor MSE for Maha Illuppallama: {mse_m_xgb}')


XGBoost Regressor MSE for Vavuniya: 106.23255745286858
XGBoost Regressor MSE for Anuradhapura: 113.44587578536905
XGBoost Regressor MSE for Maha Illuppallama: 134.93988195148313
